In [2]:
# first we get the unofficial election results:

import requests
from bs4 import BeautifulSoup
import pandas as pd

# URL of the NYT webpage containing the data
url = 'https://www.nytimes.com/interactive/2025/11/04/us/elections/results-new-york-city-mayor.html'

# send a GET request to the webpage
response = requests.get(url)

# 200 status code = YES, this went well ...
if response.status_code == 200:
    soup = BeautifulSoup(response.content, 'html.parser')

    # find the correct table (adjust selector as needed)
    tables = soup.find_all('table', {'data-rid': 'NY-G-M-new_york_city-2025-11-04'})

    if len(tables) >= 2:  # check if at least 2 such tables exist
        table = tables[1]  # select the second table (it's the 2nd one on the page)

    # extract rows
    rows = []
    # tr, td, th are all html tags: <tr> = table row, <td> = table cell ...
    for row in table.find_all('tr'):
        cells = row.find_all(['td', 'th'])  # het all cells
        row_data = [cell.text.strip() for cell in cells]
        if row_data:  # skip empty rows
            rows.append(row_data)

    # use the first row as headers and remaining rows as data
    headers = rows[0]  # first row as column names
    data = rows[1:]  # all other rows as data

    data = data[:-1]
    # chop off the last row, it was saying "See fewer" from the page

    # create DataFrame
    df = pd.DataFrame(data, columns=headers)

    # display the DataFrame
    print(df)

    # save as JSON
    json_file_path = '/content/mayor_results.json'  # Update path if needed
    df.to_json(json_file_path, orient='records', indent=4)

    print(f"Data saved to {json_file_path}")

else:
    print(f"Failed to retrieve the webpage. Status code: {response.status_code}")




           Neighborhood       Margin Mamdani Cuomo Total votes
0       Upper East Side    Cuomo +24     36%   60%      90,390
1       Upper West Side   Mamdani +5     51%   45%      85,951
2    Bedford-Stuyvesant  Mamdani +57     77%   20%      48,030
3               Astoria  Mamdani +39     66%   27%      45,362
4         Crown Heights  Mamdani +43     71%   27%      41,958
..                  ...          ...     ...   ...         ...
223                NoHo     Cuomo +6     46%   52%         976
224     Bronx Park East  Mamdani +31     63%   32%         932
225      Fort Wadsworth    Cuomo +11     33%   44%         855
226        Vinegar Hill  Mamdani +41     69%   27%         822
227       Randall Manor  Mamdani +15     51%   36%         444

[228 rows x 5 columns]
Data saved to /content/mayor_results.json
